In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
import numpy as np

from nsc.api import utils
from nsc.data import preprocessing

from spell_checking import BENCHMARK_DIR

In [14]:
benchmarks = ["wikidump/realistic", "wikidump/artificial", "bookcorpus/realistic", "bookcorpus/artificial"]

rand = np.random.default_rng(22)
samples_per_benchmark = 200
correct_lines = []
corrupt_lines = []
for benchmark in sorted(benchmarks):
    corrupt = utils.load_text_file(os.path.join(BENCHMARK_DIR, "test", "sec", benchmark, "corrupt.txt"))
    correct = utils.load_text_file(os.path.join(BENCHMARK_DIR, "test", "sec", benchmark, "correct.txt"))
    indices = rand.permutation(len(corrupt))[:samples_per_benchmark]
    correct_lines.extend([correct[idx] for idx in indices])
    corrupt_lines.extend([corrupt[idx] for idx in indices])
utils.save_text_file(os.path.join(BENCHMARK_DIR, "test", "runtime.correct.txt"), correct_lines)
utils.save_text_file(os.path.join(BENCHMARK_DIR, "test", "runtime.corrupt.txt"), corrupt_lines)

In [15]:
output_dir = os.path.join(BENCHMARK_DIR, "test", "sec", "whitespace")

In [16]:
ins_noise_levels = [0.025, 0.05, 0.1]
del_noise_levels = [0.125, 0.25, 0.5]
level_names = ["low", "medium", "high"]
for i, (ins_noise, del_noise, level_name) in enumerate(zip(ins_noise_levels, del_noise_levels, level_names)):
    cfg = preprocessing.WhitespaceNoiseConfig(no_whitespace_p=0, full_whitespace_p=0, insert_whitespace_p=ins_noise, delete_whitespace_p=del_noise)
    whitespace_noise = preprocessing.WhitespaceNoise(cfg=cfg, seed=22 + i)
    
    corrupt_noised, _, _ = whitespace_noise.apply(
        corrupt_lines,
        corrupt_lines,
        [{}] * len(runtime_benchmark_corrupt)
    )
    
    
    corrupt_output_file = os.path.join(output_dir, level_name, "corrupt.txt")
    correct_output_file = os.path.join(output_dir, level_name, "correct.txt")
    
    utils.save_text_file(corrupt_output_file, corrupt_noised)
    utils.save_text_file(correct_output_file, correct_lines)